# 第五题（选做）：请你完成带有后剪枝的决策树

## 实验内容
1. 实现带有后剪枝的决策树
2. 数据集随意
3. 最后对比剪枝和不剪枝的差别

In [1]:
# YOUR CODE HERE
# 导入类库
import pandas as pd
import numpy as np
loans = pd.read_csv('data/lendingclub/lending-club-data.csv', low_memory=False)
loans['safe_loans'] = loans['bad_loans'].apply(lambda x : +1 if x==0 else -1)
del loans['bad_loans']

In [2]:
features = ['grade',              # grade of the loan
            'term',               # the term of the loan
            'home_ownership',     # home_ownership status: own, mortgage or rent
            'emp_length',         # number of years of employment
           ]
target = 'safe_loans'
loans = loans[features + [target]]

In [3]:
from sklearn.utils import shuffle
loans = shuffle(loans, random_state = 34)
split_line1 = int(len(loans) * 0.6)
split_line2 = int(len(loans) * 0.8)
train_data = loans.iloc[: split_line1]
validation_data = loans.iloc[split_line1: split_line2]
test_data = loans.iloc[split_line2:]

In [4]:
def one_hot_encoding(data, features_categorical):
    '''
    Parameter
    ----------
    data: pd.DataFrame
    
    features_categorical: list(str)
    '''
    
    # 对所有的离散特征遍历
    for cat in features_categorical:
        
        # 对这列进行one-hot编码，前缀为这个变量名
        one_encoding = pd.get_dummies(data[cat], prefix = cat)
        
        # 将生成的one-hot编码与之前的dataframe拼接起来
        data = pd.concat([data, one_encoding],axis=1)
        
        # 删除掉原始的这列离散特征
        del data[cat]
    
    return data

In [5]:
train_data = one_hot_encoding(train_data, features)

In [6]:
one_hot_features = train_data.columns.tolist()
one_hot_features.remove(target)

In [7]:
validation_tmp = one_hot_encoding(validation_data, features)
validation_data = pd.DataFrame(columns = train_data.columns)
for feature in train_data.columns:
    if feature in validation_tmp:
        validation_data[feature] = validation_tmp[feature].copy()
    else:
        validation_data[feature] = np.zeros(len(validation_tmp), dtype = 'uint8')

In [8]:
test_data_tmp = one_hot_encoding(test_data, features)
test_data = pd.DataFrame(columns = train_data.columns)
for feature in train_data.columns:
    if feature in test_data_tmp.columns:
        test_data[feature] = test_data_tmp[feature].copy()
    else:
        test_data[feature] = np.zeros(test_data_tmp.shape[0], dtype = 'uint8')

In [9]:
print(train_data.shape, validation_data.shape, test_data.shape)

((73564, 25), (24521, 25), (24522, 25))


In [10]:
def information_entropy(labels_in_node):
    '''
    求当前结点的信息熵
    
    Parameter
    ----------
    labels_in_node: np.ndarray, 如[-1, 1, -1, 1, 1]
    
    Returns
    ----------
    float: information entropy
    '''
    # 统计样本总个数
    num_of_samples = labels_in_node.shape[0]
    
    if num_of_samples == 0:
        return 0
    
    # 统计出标记为1的个数
    num_of_positive = len(labels_in_node[labels_in_node == 1])
    
    # 统计出标记为-1的个数
    num_of_negative = len(labels_in_node[labels_in_node == -1])                                                                    # YOUR CODE HERE
    
    # 统计正例的概率
    prob_positive = num_of_positive*1.0 / num_of_samples
    
    # 统计负例的概率
    prob_negative = num_of_negative*1.0 / num_of_samples                                                                      # YOUR CODE HERE
    
    if prob_positive == 0:
        positive_part = 0
    else:
        positive_part = prob_positive * np.log2(prob_positive)
    
    if prob_negative == 0:
        negative_part = 0
    else:
        negative_part = prob_negative * np.log2(prob_negative)
    
    return - ( positive_part + negative_part )

In [11]:
def compute_information_gains(data, features, target, annotate = False):
    '''
    计算所有特征的信息增益
    
    Parameter
    ----------
        data: pd.DataFrame，传入的样本，带有特征和标记的dataframe
        
        features: list(str)，特征名组成的list
        
        target: str, 标记(label)的名字
        
        annotate, boolean，是否打印所有特征的信息增益值，默认为False
        
    Returns
    ----------
        information_gains: dict, key: str, 特征名
                                 value: float，信息增益
    '''
    
    # 我们将每个特征划分的信息增益值存储在一个dict中
    # 键是特征名，值是信息增益值
    information_gains = dict()
    
    # 对所有的特征进行遍历，使用信息增益对每个特征进行计算
    for feature in features:
        
        # 左子树保证所有的样本的这个特征取值为0
        left_split_target = data[data[feature] == 0][target]
        
        # 右子树保证所有的样本的这个特征取值为1
        right_split_target =  data[data[feature] == 1][target]
            
        # 计算左子树的信息熵
        left_entropy = information_entropy(left_split_target)
        
        # 计算左子树的权重
        left_weight = len(left_split_target)*1.0 / (len(left_split_target) + len(right_split_target))

        # 计算右子树的信息熵
        right_entropy = information_entropy(right_split_target)                                                                # YOUR CODE HERE
        
        # 计算右子树的权重
        right_weight = len(right_split_target)*1.0 / (len(left_split_target) + len(right_split_target))                                                                 # YOUR CODE HERE
        
        # 计算当前结点的信息熵
        current_entropy = information_entropy(data[target])
            
        # 计算使用当前特征划分的信息增益
        gain = current_entropy-(left_weight*left_entropy+right_weight*right_entropy)                                                                         # YOUR CODE HERE
        
        # 将特征名与增益值以键值对的形式存储在information_gains中
        information_gains[feature] = gain
        
        if annotate:
            print(" ", feature, gain)
            
    return information_gains

In [12]:
def compute_information_gain_ratios(data, features, target, annotate = False):
    '''
    计算所有特征的信息增益率并保存起来
    
    Parameter
    ----------
    data: pd.DataFrame, 带有特征和标记的数据
    
    features: list(str)，特征名组成的list
    
    target: str， 特征的名字
    
    annotate: boolean, default False，是否打印注释
    
    Returns
    ----------
    gain_ratios: dict, key: str, 特征名
                       value: float，信息增益率
    '''
    
    gain_ratios = dict()
    
    # 对所有的特征进行遍历，使用当前的划分方法对每个特征进行计算
    for feature in features:
        
        # 左子树保证所有的样本的这个特征取值为0
        left_split_target = data[data[feature] == 0][target]
        
        # 右子树保证所有的样本的这个特征取值为1
        right_split_target =  data[data[feature] == 1][target]
            
        # 计算左子树的信息熵
        left_entropy = information_entropy(left_split_target)
        
        # 计算左子树的权重
        left_weight = len(left_split_target)*1.0 / (len(left_split_target) + len(right_split_target))

        # 计算右子树的信息熵
        right_entropy = information_entropy(right_split_target)                                                                    # YOUR CODE HERE
        
        # 计算右子树的权重
        right_weight = len(right_split_target)*1.0 / (len(left_split_target) + len(right_split_target))                                                                     # YOUR CODE HERE
        
        # 计算当前结点的信息熵
        current_entropy = information_entropy(data[target])
        
        # 计算当前结点的信息增益
        
        gain = current_entropy-(left_weight*left_entropy+right_weight*right_entropy)                                                                             # YOUR CODE HERE
        
        # 计算IV公式中，当前特征为0的值
        if left_weight == 0:
            left_IV = 0
        else:
            left_IV = left_weight * np.log2(left_weight)                                                                      # YOUR CODE HERE
        
        # 计算IV公式中，当前特征为1的值
        if right_weight == 0:
            right_IV = 0
        else:
            right_IV = right_weight * np.log2(right_weight)                                                                     # YOUR CODE HERE
        
        # IV 等于所有子树IV之和的相反数
        IV = - (left_IV + right_IV)
            
        # 计算使用当前特征划分的信息增益率
        # 这里为了防止IV是0，导致除法得到np.inf（无穷），在分母加了一个很小的小数
        gain_ratio = gain*1.0 / (IV + np.finfo(np.longdouble).eps)
        
        # 信息增益率的存储
        gain_ratios[feature] = gain_ratio
        
        if annotate:
            print(" ", feature, gain_ratio)
            
    return gain_ratios

In [13]:
def gini(labels_in_node):
    '''
    计算一个结点内样本的基尼指数
    
    Paramters
    ----------
    label_in_data: np.ndarray, 样本的标记，如[-1, -1, 1, 1, 1]
    
    Returns
    ---------
    gini: float，基尼指数
    '''
    
    # 统计样本总个数
    num_of_samples = labels_in_node.shape[0]
    
    if num_of_samples == 0:
        return 0
    
    # 统计出1的个数
    num_of_positive = len(labels_in_node[labels_in_node == 1])
    
    # 统计出-1的个数
    num_of_negative = len(labels_in_node[labels_in_node == -1])                                                  # YOUR CODE HERE
    
    # 统计正例的概率
    prob_positive = num_of_positive*1.0 / num_of_samples
    
    # 统计负例的概率
    prob_negative = 1-prob_positive                                                  # YOUR CODE HERE
    
    # 计算基尼值
    gini = prob_positive*prob_negative+prob_negative*prob_positive                                                            # YOUR CODE HERE
    
    return gini

In [14]:
def compute_gini_indices(data, features, target, annotate = False):
    '''
    计算使用各个特征进行划分时，各特征的基尼指数
    
    Parameter
    ----------
    data: pd.DataFrame, 带有特征和标记的数据
    
    features: list(str)，特征名组成的list
    
    target: str， 特征的名字
    
    annotate: boolean, default False，是否打印注释
    
    Returns
    ----------
    gini_indices: dict, key: str, 特征名
                       value: float，基尼指数
    '''
    
    gini_indices = dict()
    # 对所有的特征进行遍历，使用当前的划分方法对每个特征进行计算
    for feature in features:
        # 左子树保证所有的样本的这个特征取值为0
        left_split_target = data[data[feature] == 0][target]
        
        # 右子树保证所有的样本的这个特征取值为1
        right_split_target =  data[data[feature] == 1][target]
            
        # 计算左子树的基尼值
        left_gini = gini(left_split_target)
        
        # 计算左子树的权重
        left_weight = len(left_split_target)*1.0 / (len(left_split_target) + len(right_split_target))

        # 计算右子树的基尼值
        right_gini =  gini(right_split_target)                                                             # YOUR CODE HERE
        
        # 计算右子树的权重
        right_weight = len(right_split_target)*1.0 / (len(left_split_target) + len(right_split_target))                                                            # YOUR CODE HERE
        
        # 计算当前结点的基尼指数
        gini_index = left_gini * left_weight + right_gini * right_weight                                                             # YOUR CODE HERE
        
        # 存储
        gini_indices[feature] = gini_index
        
        if annotate:
            print(" ", feature, gini_index)
            
    return gini_indices

In [15]:
def best_splitting_feature(data, features, target, criterion = 'gini', annotate = False):
    '''
    给定划分方法和数据，找到最优的划分特征
    
    Parametersa
    ----------
    data: pd.DataFrame, 带有特征和标记的数据
    
    features: list(str)，特征名组成的list
    
    target: str， 特征的名字
    
    criterion: str, 使用哪种指标，三种选项: 'information_gain', 'gain_ratio', 'gini'
    
    annotate: boolean, default False，是否打印注释
    
    Returns
    ----------
    best_feature: str, 最佳的划分特征的名字
    
    '''
    if criterion == 'information_gain':
        if annotate:
            print('using information gain')
        
        # 得到当前所有特征的信息增益
        information_gains = compute_information_gains(data, features, target, annotate)
    
        # information_gains是一个dict类型的对象，我们要找值最大的那个元素的键是谁
        # 根据这些特征和他们的信息增益，找到最佳的划分特征
        best_feature = max(information_gains,key=information_gains.get)                                                                     # YOUR CODE HERE
        
        return best_feature

    elif criterion == 'gain_ratio':
        if annotate:
            print('using information gain ratio')
        
        # 得到当前所有特征的信息增益率
        gain_ratios = compute_information_gain_ratios(data, features, target, annotate)
    
        # 根据这些特征和他们的信息增益率，找到最佳的划分特征
        best_feature = max(gain_ratios,key=gain_ratios.get)                                                                     # YOUR CODE HERE

        return best_feature
    
    elif criterion == 'gini':
        if annotate:
            print('using gini')
        
        # 得到当前所有特征的基尼指数
        gini_indices = compute_gini_indices(data, features, target, annotate)
        
        # 根据这些特征和他们的基尼指数，找到最佳的划分特征
        best_feature = min(gini_indices,key=gini_indices.get)                                                                       # YOUR CODE HERE

        return best_feature
    else:
        raise Exception("传入的criterion不合规!", criterion)

In [16]:
def intermediate_node_num_mistakes(labels_in_node):
    '''
    求树的结点中，样本数少的那个类的样本有多少，比如输入是[1, 1, -1, -1, 1]，返回2
    
    Parameter
    ----------
    labels_in_node: np.ndarray, pd.Series
    
    Returns
    ----------
    int：个数
    
    '''
    # 如果传入的array为空，返回0
    if len(labels_in_node) == 0:
        return 0
    
    # 统计1的个数
    num_of_one = len(labels_in_node[labels_in_node == 1])                                                               # YOUR CODE HERE
    
    # 统计-1的个数
    num_of_minus_one = len(labels_in_node[labels_in_node == -1])                                                               # YOUR CODE HERE
    
    return num_of_one if num_of_minus_one > num_of_one else num_of_minus_one

In [17]:
def majority_class(labels_in_node):
    '''
        求树的结点中，样本数多的那个类是什么
    '''
    # 如果传入的array为空，返回0
    if len(labels_in_node) == 0:
        return 0
    
    # 统计1的个数
    num_of_one = len(labels_in_node[labels_in_node == 1])                                                              # YOUR CODE HERE
    
    # 统计-1的个数
    num_of_minus_one = len(labels_in_node[labels_in_node == -1])                                                        # YOUR CODE HERE
    
    return 1 if num_of_minus_one < num_of_one else -1

In [18]:
def create_leaf(target_values):
    '''
    计算出当前叶子结点的标记是什么，并且将叶子结点信息保存在一个dict中
    
    Parameter:
    ----------
    target_values: pd.Series, 当前叶子结点内样本的标记

    Returns:
    ----------
    leaf: dict，表示一个叶结点，
            leaf['splitting_features'], None，叶结点不需要划分特征
            leaf['left'], None，叶结点没有左子树
            leaf['right'], None，叶结点没有右子树
            leaf['is_leaf'], True, 是否是叶子结点
            leaf['prediction'], int, 表示该叶子结点的预测值
    '''
    # 创建叶子结点
    leaf = {'splitting_feature' : None,
            'left' : None,
            'right' : None,
            'is_leaf': True}
   
    # 数结点内-1和+1的个数
    num_ones = len(target_values[target_values == +1])
    num_minus_ones = len(target_values[target_values == -1])    

    # 叶子结点的标记使用少数服从多数的原则，为样本数多的那类的标记，保存在 leaf['prediction']
    if num_ones > num_minus_ones:
        leaf['prediction'] = 1
    else:
        leaf['prediction'] = -1

    # 返回叶子结点
    return leaf

In [19]:
def decision_tree_create(training_data, validation_data, features, target, criterion = 'gain_ratios', pre_pruning = False, current_depth = 0, max_depth = 10, annotate = False):
    '''
    Parameter:
    ----------
    trianing_data: pd.DataFrame, 数据

    features: iterable, 特征组成的可迭代对象

    target: str, 标记的名字

    criterion: 'str', 特征划分方法

    current_depth: int, 当前深度

    max_depth: int, 树的最大深度

    Returns:
    ----------
    dict, dict['is_leaf']          : False, 当前顶点不是叶子结点
          dict['prediction']       : None, 不是叶子结点就没有预测值
          dict['splitting_feature']: splitting_feature, 当前结点是使用哪个特征进行划分的
          dict['left']             : dict
          dict['right']            : dict
    '''
    if criterion not in ['information_gain', 'gain_ratio', 'gini']:
        raise Exception("传入的criterion不合规!", criterion)
    
    # 复制一份特征，存储起来，每使用一个特征进行划分，我们就删除一个
    remaining_features = features[:]
    
    # 取出标记值
    target_values = training_data[target]
    validation_values = validation_data[target]
    print("-" * 50)
    print("Subtree, depth = %s (%s data points)." % (current_depth, len(target_values)))

    # 终止条件1
    # 如果当前结点内所有样本同属一类，即这个结点中，各类别样本数最小的那个等于0
    # 使用前面写的intermediate_node_num_mistakes来完成这个判断
    if intermediate_node_num_mistakes(target_values) == 0:                                                                                       # YOUR CODE HERE
        print("Stopping condition 1 reached.")
        return create_leaf(target_values)   # 创建叶子结点
    
    # 终止条件2
    # 如果已经没有剩余的特征可供分割
    if len(remaining_features) == 0:                                                                                       # YOUR CODE HERE
        print("Stopping condition 2 reached.")
        return create_leaf(target_values)   # 创建叶子结点
    
    # 终止条件3
    # 如果已经到达了我们要求的最大深度
    if current_depth == max_depth:                                                                                       # YOUR CODE HERE
        print("Reached maximum depth. Stopping for now.")
        return create_leaf(target_values)   # 创建叶子结点
    
    # 找到最优划分特征
    # 使用best_splitting_feature这个函数
    splitting_feature = best_splitting_feature(training_data, features, target, criterion)                                                                      # YOUR CODE HERE
    
    # 使用我们找到的最优特征将数据划分成两份
    # 左子树的数据
    left_split = training_data[training_data[splitting_feature] == 0]
    
    # 右子树的数据
    right_split = training_data[training_data[splitting_feature] == 1]                                                                            # YOUR CODE HERE
    
    # 使用这个最优特征对验证集进行划分
    validation_left_split = validation_data[validation_data[splitting_feature] == 0]
    validation_right_split = validation_data[validation_data[splitting_feature] == 1]
    
    # 如果使用预剪枝，需要判断在验证集上的精度是否提升了
    if pre_pruning:
        # 首先计算不划分的时候的在验证集上的精度，也就是当前结点为叶子结点
        # 统计当前结点样本中，样本数多的那个类(majority class)
        true_class = majority_class(validation_values)                                                                         # YOUR CODE HERE

        # 判断验证集在不划分时的精度，分母加eps是因为，有可能在划分的时候，验证集的样本数为0
        acc_without_splitting = len(validation_values[validation_values == true_class]) / (len(validation_values) + np.finfo(np.longdouble).eps)

        # 对当前结点进行划分，统计划分后，左子树的majority class
        left_true_class = majority_class(left_split[target])

        # 对当前结点进行划分，统计右子树的majority class
        right_true_class = majority_class(right_split[target])                                                                   # YOUR CODE HERE

        # 统计验证集左子树中有多少样本是左子树的majority class
        vali_left_num_of_majority = len(validation_left_split[validation_left_split[target] == left_true_class])

        # 统计验证集右子树中有多少样本是右子树的majority class
        vali_right_num_of_majority = len(validation_right_split[validation_right_split[target] == right_true_class])                                                         # YOUR CODE HERE

        # 计算划分后的精度
        acc_with_splitting = (vali_left_num_of_majority + vali_right_num_of_majority) / (len(validation_data) + np.finfo(np.longdouble).eps)

        if annotate == True:
            print('acc before splitting: %.3f'%(acc_without_splitting))
            print('acc after splitting: %.3f'%(acc_with_splitting))

        # 如果划分后的精度小于等于划分前的精度，那就不划分，当前结点直接变成叶子结点
        # 否则继续划分，创建左右子树
        if acc_with_splitting < acc_without_splitting:
            print('Pre-Pruning')
            # 创建叶子结点
            return create_leaf(target_values)                                                                          # YOUR CODE HERE
    
    # 从剩余特征中删除掉当前这个特征
    remaining_features.remove(splitting_feature)
    
    print("Split on feature %s. (%s, %s)" % (\
                      splitting_feature, len(left_split), len(right_split)))
    
    # 如果使用当前的特征，将所有的样本都划分到一棵子树中，那么就直接将这棵子树变成叶子结点
    # 判断左子树是不是“完美”的
    if len(left_split) == len(training_data):
        print("Creating leaf node.")
        return create_leaf(left_split[target])
    
    # 判断右子树是不是“完美”的
    if len(right_split) == len(training_data):
        print("Creating right node.")
        return create_leaf(right_split[target])                                                                                # YOUR CODE HERE

    # 递归地创建左子树，需要传入验证集的左子树
    left_tree = decision_tree_create(left_split, validation_left_split, remaining_features, target, criterion, pre_pruning, current_depth + 1, max_depth, annotate)
    
    # 递归地创建右子树，需要传入验证集的右子树
    
    right_tree = decision_tree_create(right_split, validation_right_split, remaining_features, target, criterion, pre_pruning, current_depth + 1, max_depth, annotate)                                                                             # YOUR CODE HERE

    return {'is_leaf'          : False, 
            'prediction'       : None,
            'splitting_feature': splitting_feature,
            'left'             : left_tree, 
            'right'            : right_tree}

In [20]:
def post_pruning(tree,  training_data, validation_data, features, target):
        # 寻找叶子节点
        if tree['left'] is None or tree['right'] is None:
            return
        
        # 取出标记值
        target_values = training_data[target]
        validation_values = validation_data[target]
        splitting_feature = tree["splitting_feature"]
        
        # 左分支数据
        left_split = training_data[training_data[splitting_feature] == 0]
        # 右分支数据
        right_split = training_data[training_data[splitting_feature] == 1]
        
        # 使用这个最优特征对验证集进行划分
        validation_left_split = validation_data[validation_data[splitting_feature] == 0]
        validation_right_split = validation_data[validation_data[splitting_feature] == 1]
        
        # 后序遍历
        post_pruning(tree['left'], left_split, validation_left_split, features, target)
        post_pruning(tree["right"], right_split, validation_right_split, features, target)
        
        # 统计当前结点样本中，样本数多的那个类(majority class)
        true_class = majority_class(target_values)
        acc_without_splitting = len(validation_values[validation_values == true_class]) / (len(validation_values) + np.finfo(np.longdouble).eps)
        
        # 对当前结点进行划分，统计划分后，左子树的majority class
        left_true_class = majority_class(left_split[target])
        
        # 对当前结点进行划分，统计划分后，右子树的majority class
        right_true_class = majority_class(right_split[target])
        
        # 统计验证集左子树中有多少样本是左子树的majority class
        vali_left_num_of_majority = len(validation_left_split[validation_left_split[target] == left_true_class])
        
        # 统计验证集右子树中有多少样本是右子树的majority class
        vali_right_num_of_majority = len(validation_right_split[validation_right_split[target] == right_true_class])  # YOUR CODE HERE
        
        # 计算划分后的精度
        acc_with_splitting = (vali_left_num_of_majority + vali_right_num_of_majority) / (len(validation_data) + np.finfo(np.longdouble).eps)
        
        # 后剪枝核心(若减枝前小于减枝后则减枝)
        if acc_with_splitting < acc_without_splitting:
            tree['is_leaf'] = True
            tree['prediction'] = true_class
            tree['left'] = None
            tree['right'] = None

In [21]:
def classify(tree, x, annotate = False):
    '''
    递归的进行预测，一次只能预测一个样本
    
    Parameters
    ----------
    tree: dict
    
    x: pd.Series，待预测的样本
    
    annotate： boolean, 是否显示注释
    
    Returns
    ----------
    返回预测的标记
    '''
    if tree['is_leaf']:
        if annotate:
            print ("At leaf, predicting %s" % tree['prediction'])
        return tree['prediction']
    else:
        split_feature_value = x[tree['splitting_feature']]
        if annotate:
             print ("Split on %s = %s" % (tree['splitting_feature'], split_feature_value))
        if split_feature_value == 0:
            return classify(tree['left'], x, annotate)
        else:
            return classify(tree['right'], x, annotate)

In [22]:
def predict(tree, data):
    '''
    按行遍历data，对每个样本进行预测，将值存在prediction中，最后返回np.ndarray
    
    Parameter
    ----------
    tree: dict, 模型
    
    data: pd.DataFrame, 数据
    
    Returns
    ----------
    predictions：np.ndarray, 模型对这些样本的预测结果
    '''
    list = []
    for i in range(len(data)):
        list.append(classify(tree,data.iloc[i]))
    # YOUR CODE HERE
    predictions = np.array(list) # 长度和data一样
    print(data['safe_loans'].shape)
    print(predictions.shape)
    return predictions

In [23]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [24]:
# 导入树形图
from pyecharts import Tree

In [25]:
def generate_echarts_data(tree):
    
    # 当前顶点的dict
    value = dict()
    
    # 如果传入的tree已经是叶子结点了
    if tree['is_leaf'] == True:
        
        # 它的value就设置为预测的标记
        value['value'] = tree['prediction']
        
        # 它的名字就叫"label: 标记"
        value['name'] = 'label: %s'%(tree['prediction'])
        
        # 直接返回这个dict即可
        return value
    
    # 如果传入的tree不是叶子结点，名字就叫当前这个顶点的划分特征，子树是一个list
    # 分别增加左子树和右子树到children中
    value['name'] = tree['splitting_feature']
    value['children'] = [generate_echarts_data(tree['left']), generate_echarts_data(tree['right'])]
    return value

In [26]:
my_tree = decision_tree_create(train_data, validation_data, one_hot_features, target, criterion = 'gain_ratio', pre_pruning = False, current_depth = 0, max_depth = 6, annotate = False)

--------------------------------------------------
Subtree, depth = 0 (73564 data points).
Split on feature grade_F. (71229, 2335)
--------------------------------------------------
Subtree, depth = 1 (71229 data points).
Split on feature grade_A. (57869, 13360)
--------------------------------------------------
Subtree, depth = 2 (57869 data points).
Split on feature grade_G. (57232, 637)
--------------------------------------------------
Subtree, depth = 3 (57232 data points).
Split on feature grade_E. (51828, 5404)
--------------------------------------------------
Subtree, depth = 4 (51828 data points).
Split on feature grade_D. (40326, 11502)
--------------------------------------------------
Subtree, depth = 5 (40326 data points).
Split on feature term_ 60 months. (34566, 5760)
--------------------------------------------------
Subtree, depth = 6 (34566 data points).
Reached maximum depth. Stopping for now.
--------------------------------------------------
Subtree, depth = 6 (57

Split on feature home_ownership_RENT. (6, 11)
--------------------------------------------------
Subtree, depth = 6 (6 data points).
Reached maximum depth. Stopping for now.
--------------------------------------------------
Subtree, depth = 6 (11 data points).
Reached maximum depth. Stopping for now.
--------------------------------------------------
Subtree, depth = 5 (1 data points).
Stopping condition 1 reached.
--------------------------------------------------
Subtree, depth = 3 (1719 data points).
Split on feature home_ownership_OTHER. (1717, 2)
--------------------------------------------------
Subtree, depth = 4 (1717 data points).
Split on feature emp_length_3 years. (1577, 140)
--------------------------------------------------
Subtree, depth = 5 (1577 data points).
Split on feature home_ownership_RENT. (904, 673)
--------------------------------------------------
Subtree, depth = 6 (904 data points).
Reached maximum depth. Stopping for now.
---------------------------------

In [27]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
print("gain_ratio:")
prediction1 = predict(my_tree,test_data)
print("without:")
print(accuracy_score(test_data['safe_loans'],prediction1))
print(precision_score(test_data['safe_loans'],prediction1))
print(recall_score(test_data['safe_loans'],prediction1))
print(f1_score(test_data['safe_loans'],prediction1))

gain_ratio:
(24522,)
(24522,)
without:
0.8088247288149417
0.8097397556890141
0.9984383658253992
0.8942429164410755


In [28]:
data = generate_echarts_data(my_tree)

In [29]:
tree = Tree(width=800, height=400)
tree.add("",
         [data],
         tree_collapse_interval=5,
         tree_top="15%",
         tree_right="20%",
         tree_symbol = 'rect',
         tree_symbol_size = 20,
         )
tree.render()
tree

In [30]:
post_pruning(my_tree, train_data, validation_data, features, target)

In [31]:
prediction2 = predict(my_tree,test_data)
print("with:")
print(accuracy_score(test_data['safe_loans'],prediction2))
print(precision_score(test_data['safe_loans'],prediction2))
print(recall_score(test_data['safe_loans'],prediction2))
print(f1_score(test_data['safe_loans'],prediction2))

(24522,)
(24522,)
with:
0.8093956447271837
0.8094946775969656
0.9998488741121354
0.894658553076403


In [32]:
data = generate_echarts_data(my_tree)

In [33]:
tree = Tree(width=800, height=400)
tree.add("",
         [data],
         tree_collapse_interval=5,
         tree_top="15%",
         tree_right="20%",
         tree_symbol = 'rect',
         tree_symbol_size = 20,
         )
tree.render()
tree

模型|精度|查准率|查全率|F1
-|-|-|-|-
有预剪枝| 0.8094 | 0.8095 | 0.999 | 0.8947
无预剪枝| 0.8088 | 0.8097 | 0.9984| 0.8942